In [ ]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from skimage.data import coffee
from scipy.ndimage import map_coordinates
from eo_tools.S1.util import remap

In [ ]:


siz_in = 128
# img = np.random.rand(siz_in, siz_in)
img = coffee().mean(2).astype("float64") 
# img = np.zeros((64,64))
# img[32,32] = 1

naz, nrg = img.shape

siz_out_1 = 1500
siz_out_2 = 22000
rr = np.random.rand(siz_out_1, siz_out_2) * (naz - 1)
cc = np.random.rand(siz_out_1, siz_out_2) * (nrg - 1)


# rr, cc = np.mgrid[:naz, :nrg].astype("float64")

theta = np.pi/6
rr = rr*np.cos(theta) - cc*np.sin(theta)
cc = rr*np.sin(theta) + cc*np.cos(theta)

kernel = "bicubic6"

if kernel == "nearest":
    order = 0
if kernel == "bilinear":
    order = 1
if kernel == "bicubic6":
    order = 3

# re_mc = 
# %lprun -u1 -f map_coordinates map_coordinates(img, (rr, cc), order=order, prefilter=False)
%timeit map_coordinates(img, (rr, cc), order=order, prefilter=False)

# re = 
%timeit remap(img, rr, cc, kernel)

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(re, interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)
plt.gray()
plt.figure(figsize=(10, 10))
plt.imshow(re_mc, interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)
plt.gray()
plt.figure(figsize=(10, 10))
plt.imshow(re_mc-re, interpolation="none")
plt.colorbar(fraction=0.046, pad=0.04)
plt.gray()

In [ ]:
# def _ker_cub6(x):
#     a = -0.5
#     b = 0.5
#     ax = np.abs(x)
#     ax2 = ax**2
#     ax3 = ax**3
#     if ax < 1:
#         return (a + b + 2) * ax3 - (a + b + 3) * ax2 + 1
#     elif (ax >= 1) & (ax < 2):
#         return a * ax3 - (5 * a - b) * ax2 + (8 * a - 3 * b) * ax - (4 * a - 2 * b)
#     elif (ax >= 2) & (ax < 3):
#         return b * ax3 - 8 * b * ax2 + 21 * b * ax - 18 * b
#     else:
        # return 0.0

def _ker_cub6(x):
    ax = np.abs(x)
    ax2 = ax**2
    ax3 = ax**3
    if ax < 1:
        return 4 * ax3 / 3 - 7 * ax2 / 3 + 1
    elif (ax >= 1) & (ax < 2):
        return -7 * ax3 / 12 + 3 * ax2 - 59 * ax / 12 + 5 / 2
    elif (ax >= 2) & (ax < 3):
        return ax3 / 12 - 2 * ax2 / 3 + 7 * ax / 4 - 1.5
    else:
        return 0.0

In [ ]:
xx = np.linspace(-3,3,1000)
arr = np.array([_ker_cub6(x) for x in xx])
plt.plot(xx, arr)